# Урок 12. Модель Transformer-2

Реализовать суммаризацию текста

In [10]:
# !pip install transformers sentencepiece rouge razdel
# !wget -q https://www.dropbox.com/s/43l702z5a5i2w8j/gazeta_train.txt
# !wget -q https://www.dropbox.com/s/k2egt3sug0hb185/gazeta_val.txt
# !wget -q https://www.dropbox.com/s/3gki5n5djs9w0v6/gazeta_test.txt

In [11]:
import pandas as pd
import numpy as np
import json
import random
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge
import razdel
import torch
import razdel

def calc_lead_n_score(records, summary_col = 'summary', n=3, lower=True, nrows=1000):
    references = []
    predictions = []
    
    itr = records.iterrows() if isinstance(records, pd.DataFrame) else enumerate(records)

    for i, record in itr:
        if i >= nrows:
            break

        summary = record[summary_col]
        summary = summary if not lower else summary.lower()
        references.append(summary)

        text = record["text"]
        text = text if not lower else text.lower()
        sentences = [sentence.text for sentence in razdel.sentenize(text)]
        prediction = " ".join(sentences[:n])
        predictions.append(prediction)

    calc_scores(references, predictions)

def read_gazeta_records(file_name, shuffle=True, sort_by_date=False):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r") as r:
        for line in r:
            records.append(json.loads(line))
    if sort_by_date:
        records.sort(key=lambda x: x["date"])
    if shuffle:
        random.shuffle
    return records

def calc_scores(references, predictions, metric="all"):
    print("Count:", len(predictions))
    print("Ref:", references[-1])
    print("Hyp:", predictions[-1])

    if metric in ("bleu", "all"):
        print("BLEU: ", corpus_bleu([[r] for r in references], predictions))
    if metric in ("rouge", "all"):
        rouge = Rouge()
        scores = rouge.get_scores(predictions, references, avg=True)
        print("ROUGE: ", scores)

In [12]:
train_records = read_gazeta_records("gazeta_train.txt")
val_records = read_gazeta_records("gazeta_val.txt")
test_records = read_gazeta_records("gazeta_test.txt")

In [13]:
test_records[0]

{'url': 'https://www.gazeta.ru/science/2020/02/14_a_12960289.shtml',
 'text': 'Американское аэрокосмическое агентство NASA огласило названия четырех космических миссий, которые в скором времени могут быть выбраны для реализации и запуск которых может состояться уже в конце этого десятилетия. Эти четыре проекта стали полуфиналистами конкурса, объявленного среди американских научных команд, в котором участвовало более десяти миссий. Все они были отобраны по критериям потенциальной пользы для науки и технической осуществимости проекта. В рамках программы Discovery NASA занимается планированием миссий, которые призваны дать ответы на фундаментальные вопросы о происхождении тел Солнечной системы и возможному наличию жизни на них. «Эти выбранные миссии могут трансформировать наше восприятие некоторых из наиболее активных и сложных миров в Солнечной системе, — заявил Томас Зурбучен, помощник директора NASA по науке. — Исследование каждого из этих небесных тел поможет раскрыть секреты о том, к

In [14]:
test_records[0].keys()

dict_keys(['url', 'text', 'title', 'summary', 'date'])

In [15]:
print(min([record["date"] for record in train_records]), end=' - ')
print(max([record["date"] for record in train_records]))
print(min([record["date"] for record in val_records]), end=' - ')
print(max([record["date"] for record in val_records]))
print(min([record["date"] for record in test_records]), end=' - ')
print(max([record["date"] for record in test_records]))

2010-06-01 10:35:49 - 2019-05-31 23:56:26
2019-06-01 08:30:00 - 2019-09-30 23:11:23
2019-10-01 08:23:02 - 2020-03-23 22:16:23


В качестве метрик будем использовать BLEU и ROUGE.

In [16]:
calc_lead_n_score(test_records, n=1)

Count: 1000
Ref: телеканал «спас» запускает реалити-шоу «остров», участникам которого предстоит месяц жить и работать в нило-столобенской пустыни на озере селигер. организаторы отметили, что это беспрецедентный подобный проект на телевидении. участникам шоу будет, где поработать — в монастыре работают свечной, молочный и столярный цеха, есть коровник, конюшня, пасека.
Hyp: православный телеканал «спас», учредителем которого является московская патриархия, запускает реалити-шоу «остров», участникам которого предстоит месяц жить и работать в нило-столобенской пустыни на озере селигер в тверской области.
BLEU:  0.19177311186434495
ROUGE:  {'rouge-1': {'r': 0.37762764047433917, 'p': 0.22208274285774904, 'f': 0.23804097238957525}, 'rouge-2': {'r': 0.15833772153385062, 'p': 0.09647636782929753, 'f': 0.10027796832321115}, 'rouge-l': {'r': 0.34937017731940756, 'p': 0.2022959168891477, 'f': 0.21799992093276083}}


**В качестве готовой модели возьмем 'cointegrated/rut5-base-absum'.**

In [17]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
MODEL_NAME = 'cointegrated/rut5-base-absum'
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
# model.cuda();
model.eval();

def summarize(
    text, n_words=None, compression=None,
    max_length=1000, num_beams=3, do_sample=False, repetition_penalty=10.0, 
    **kwargs
):

    if n_words:
        text = '[{}] '.format(n_words) + text
    elif compression:
        text = '[{0:.1g}] '.format(compression) + text
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    with torch.inference_mode():
        out = model.generate(
            **x, 
            max_length=max_length, num_beams=num_beams, 
            do_sample=do_sample, repetition_penalty=repetition_penalty, 
            **kwargs
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [18]:
sum_pred = summarize(test_records[1]['text'], len(test_records[1]['summary'].split()))

In [19]:
sum_pred

'В Бурятии прошел праздничный концерт «Танцуют все!» на телеканале «Россия»'

In [20]:
test_records[1]['summary']

'25 и 26 февраля в Кремлевском дворце съездов праздновали Сагаалган — Восточный Новый год. Бурятия - центр российского буддизма и один из немногих регионов страны, где новый год встречают официально дважды.'

In [21]:
df_test = pd.DataFrame(test_records)

In [22]:
df_test.head(1)

,url,text,title,summary,date
0,https://www.gazeta.ru/science/2020/02/14_a_129...,Американское аэрокосмическое агентство NASA ог...,"Венера, Ио или Тритон: куда полетит NASA","В NASA назвали четыре миссии в дальний космос,...",2020-02-14 16:39:11


In [23]:
df_test.shape

(5770, 5)

Для ускорения процесса тестирования модели уменьшим размер датасета.

In [24]:
df_mini = df_test[:10]

In [25]:
df_mini['summary_pred'] = df_mini.apply(lambda x: summarize(x['text'], len(x['summary'].split())), axis=1)

/tmp/ipykernel_671/1841284967.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mini['summary_pred'] = df_mini.apply(lambda x: summarize(x['text'], len(x['summary'].split())), axis=1)


In [26]:
df_mini.head(5)

,url,text,title,summary,date,summary_pred
0,https://www.gazeta.ru/science/2020/02/14_a_129...,Американское аэрокосмическое агентство NASA ог...,"Венера, Ио или Тритон: куда полетит NASA","В NASA назвали четыре миссии в дальний космос,...",2020-02-14 16:39:11,Американское аэрокосмическое агентство NASA об...
1,https://www.gazeta.ru/social/2020/02/28/129806...,Около 11 тысяч зрителей увидели все самое лучш...,«Люди в Бурятии очень талантливые»,25 и 26 февраля в Кремлевском дворце съездов п...,2020-02-28 10:44:13,В Бурятии прошел праздничный концерт «Танцуют ...
2,https://www.gazeta.ru/politics/2019/11/07_a_12...,7 ноября в Белоруссии прошли выборы членов сов...,Вспомнить СССР: как Лукашенко провел выборы,В Белоруссии в день годовщины Октябрьской рево...,2019-11-07 19:55:08,В Белоруссии впервые прошли выборы членов сове...
3,https://www.gazeta.ru/culture/2020/03/01/a_129...,Народная артистка РСФСР Надежда Бабкина в инте...,«Он очень переживал»: Бабкина об отношениях с ...,Народная артистка РСФСР Надежда Бабкина в инте...,2020-03-01 16:50:06,"Надежда Бабкина рассказала, как ей удалось сбр..."
4,https://www.gazeta.ru/business/2020/02/06/1294...,Депутат Верховной рады от партии «Слуга народа...,«Поддерживают Россию»: почему Киев не платит п...,Украина не должна выплачивать пенсии жителям Д...,2020-02-06 12:41:24,В Верховной Раде раскритиковали законопроект о...


In [27]:
calc_lead_n_score(df_mini, n=10)

Count: 10
Ref: работающие пенсионеры дождались индексации. это следует из уже одобренных в первом чтении поправок в конституцию. так что работающие пенсионеры смогут получить замороженную с 2016 года прибавку к пенсии. в россии их насчитывается более девяти миллионов. эксперты называют это восстановлением справедливости.
Hyp: госдума одобрила в первом чтении внесенные президентом рф владимиром путиным поправки в конституцию. в его основу легли предложения, которые президент россии владимир путин озвучил в послании федеральному собранию. одно из них — регулярная индексация, формирование системы пенсионного обеспечения граждан россии «на основе принципов всеобщности, справедливости и солидарности поколений». сопредседатель профильной рабочей группы по конституции россии талия хабриева пояснила, что поправка об индексации пенсий распространяется и на работающих пенсионеров. по словам хабриевой, само правило сформулировано без изъятий, и возможна только конкретизация порядка в федеральном 

In [28]:
calc_lead_n_score(df_mini, summary_col = 'summary_pred', n=10)

Count: 10
Ref: госдума одобрила в первом чтении поправки в конституцию россии. она закрепила, что государство готово выполнять своие обязательства
Hyp: госдума одобрила в первом чтении внесенные президентом рф владимиром путиным поправки в конституцию. в его основу легли предложения, которые президент россии владимир путин озвучил в послании федеральному собранию. одно из них — регулярная индексация, формирование системы пенсионного обеспечения граждан россии «на основе принципов всеобщности, справедливости и солидарности поколений». сопредседатель профильной рабочей группы по конституции россии талия хабриева пояснила, что поправка об индексации пенсий распространяется и на работающих пенсионеров. по словам хабриевой, само правило сформулировано без изъятий, и возможна только конкретизация порядка в федеральном законе. о том, что из нынешних поправок в конституцию проистекает и индексация пенсий работающим пенсионерам, заявил и депутат госдумы олег шеин. по его словам, увеличение выпл